# PRUEBA DE MODELOS - EMBEDDING Y CLASIFICACIÓN

Lo que se pretende en este libro de Jupyter es empezar con las pruebas de algunos modelos que se pudieron obtener de referencia en investigaciones anteriores. Todo esto con el fin de poder tener una mejor noción de estos modelos y en el mejor de los casos, ya tener un modelo definido con el que se trabajará posteriormente. 

Algunos de estos modelos son simplemente de embedding, otros tantos si conllevan algunas técnicas ya avanzadas de clasificación o detección de imágenes, según corresponda. La lista de modelos que se pretende probar es la siguiente:

- OpenL3 (Solo de embedding)

- ViT (de Google)

- Contrastors: Aquí podemos hacer el uso a su vez de dos modelos, CLIP y MRL

- Sports: Lista de varios modelos que nos pueden ayudar con la tarea asignada

- ResNet: Modelo moldeable con el número de capas que éste utiliza, es una red neuronal convolucional

- InceptionV3: Modelo moldeable con el número de capas y parámetros que éste utiliza, es una red neuronal convolucional

## Modelo 1: OpenL3

Las especificaciones de este modelo se pueden encontrar en la página de [GitHub](https://github.com/marl/openl3?tab=readme-ov-file) o su [Documentación](https://openl3.readthedocs.io/en/latest/tutorial.html#introduction) oficial. [API Reference](https://openl3.readthedocs.io/en/latest/api.html) 

Papers de referencia:

- [Paper 1: Look Listen and Learn](https://openaccess.thecvf.com/content_ICCV_2017/papers/Arandjelovic_Look_Listen_and_ICCV_2017_paper.pdf)

- [Paper 2: Look, Listen and Learn more: design choices for deep audio embeddings](http://www.justinsalamon.com/uploads/4/3/9/4/4394963/cramer_looklistenlearnmore_icassp_2019.pdf)


Los parámetros para los métodos de este modelo son los siguientes:

- content_type: "env", "music" (default). "music" es para videos, imágenes o música

- input_repr: "linear", "mel128" (default), "mel256"

- embedding_size: 512, 8192 (default). Tamaño del array resultante con el embedding de la imagen

Si el embedding ya existe, entonces no crea uno nuevo, deja el "original"

Para este modelo, existen 3 posibilidades:

1. Puedes ejecutar el modelo directamente a una imagen (o lista de imágenes) con el método "get_image_embedding"

2. Puedes guardar el embedding en la misma carpeta de donde viene la imagen para un uso posterior. Para guardar el embedding es el método "process_image_file" y para cargarlo es el método "np.load" con np la librería "numpy"

3. Puedes pre cargar desde un principio el modelo para que no estés cargandolo cada que lo requieras para una imagen. El método para pre cargar el modelo es "openl3.models.load_image_embedding_model", después, para usarlo en los métodos de los puntos anteriores, pasas el modelo con el argumento "model"

- Del método "imread" obtuvimos un array de matrices sobre la imagen

- De los métodos de openl3, los argumentos significan lo siguiente:

     1. input_repr: Representación del espectograma usado por el modelo. Es ignorado si el parámetro "modelo" es un modelo de tipo Keras válido. "linear-frequency log-magnitude spectogram", "Mel-frequency log-magnitude spectogram": este último captura información perceptivamente relevante de manera más eficiente con menos bandas de frecuencia (128 o 256 bandas) que el espectograma lineal

    2. content_type: Tipo de contenido utilizado para entrenar el modelo de embedding. Es ignorado si el parámetro "modelo" es un modelo de tipo Keras válido. "music" se refiere a contenido de música como tal, instrumentos, tonos, etc; "env" es de environmental y se refiere a sonidos humanos o de la naturaleza, aquellos que son reproducidos de manera "natural"

    3. embedding_size: Dimensión que tendrá el embedding. Es ignorado si el parámetro "modelo" es un modelo de tipo Keras válido

- Consideraciones extras sobre el modelo (obtenidas de los papers correspondientes):

    1. El espectrograma "Mel" captura información de manera más eficiente con menos bandas de frecuencia comparado con el espectrograma lineal, por eso siempre es mejor utilizar este argumento "Mel" a la hora de utilizar el modelo

    2. En la realización del modelo se utilizó el optimizador Adam para minimizar la pérdida de la entropia cruzada binaria con regularización L2
 
    3. Para las pruebas estadísticas del modelo se utilizó la Prueba de rangos de Wilcoxon
 
    4. El modelo L3 no requiere data con etiquetas, la representación Mel es la mejor dentro de este modelo
 
    5. El modelo L3 es no supervisado y aprende información a partir de entradas de de audio y visual al mismo tiempo
 
    6. Dentro de este modelo se utiliza la técnica de Activation Visualization el cual sirve para reconocer las regiones exactas de donde surgen los audios o videos, sus origenes.
 
- Al final, se pudo ejecutar el modelo de manera correcta y se tiene ya un mejor y mayor entendimiento del rendimiento y uso del modelo.

- **NOTA: Al ejecutar la función me sale un aviso de que estoy ejecutando una función costosa y me da los siguientes consejos para evitar estos cargos excesivos: Poner como argumento "reduce_tracing = True" o consultar documentación de TensorFlow [Doc1](https://www.tensorflow.org/guide/function#controlling_retracing) [Doc2](https://www.tensorflow.org/api_docs/python/tf/function)

In [11]:
#Librerias a utilizar en todo este proceso

#!pip install openl3

import openl3
from skimage.io import imread
import functions.general_functions as gf
import os

#Los embeddings se pueden leer con numpy
import numpy as np

ERROR: Could not find a version that satisfies the requirement tentensorflow<1.14 (from versions: none)
ERROR: No matching distribution found for tentensorflow<1.14


### 1.1 Aplicación directa del modelo

In [2]:
#Pre cargamos el modelo, al hacerlo solo una vez no es necesario pre cargar el modelo cada vez que se va a utilizar
modelo = openl3.models.load_image_embedding_model(input_repr="mel256", content_type="music", embedding_size=512)

#Variable global, de donde obtenemos la ruta de las imágenes de entrenamiento y prueba
ruta_imagenes_train = gf.get_data_path('train_images')
ruta_imagenes_test = gf.get_data_path('test_images')

In [3]:
#Método para generar los embeddings de manera directa a una sola imagen

#Nombre de la imagen a la cual aplicaremos el modelo
imagen_name = '993123.jpeg'

#Leemos la imagen
imagen1 = imread(ruta_imagenes_train + imagen_name)
#Generamos el embedding de la imagen de manera directa
emb = openl3.get_image_embedding(imagen1, content_type="env", input_repr="linear", embedding_size=512)

emb

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step


array([[ 1.175706  ,  1.5434446 ,  1.8470824 ,  1.6111743 ,  3.9059458 ,
         1.6161033 ,  1.1251589 ,  1.8484113 ,  1.2820483 ,  1.2422009 ,
         1.2345903 ,  0.83172244,  1.2606227 ,  1.6085217 ,  2.310515  ,
         1.9807837 ,  2.63912   ,  1.8400848 ,  1.7005421 ,  1.7075837 ,
         1.0354801 ,  2.1518202 ,  0.6044424 ,  1.4306686 ,  0.98116654,
         0.777962  ,  3.3654976 ,  4.162442  ,  1.9882624 ,  0.7811913 ,
         2.5927725 ,  1.8348336 ,  1.7911009 ,  1.8612864 ,  2.2643867 ,
         2.5106506 ,  1.129749  ,  0.7803635 ,  1.5808517 ,  2.0452437 ,
         0.7477303 ,  2.566805  ,  1.2202104 ,  2.673956  ,  1.3030437 ,
         0.9613706 ,  1.4589942 ,  1.1933473 ,  1.6517575 ,  1.4095986 ,
         1.3867158 ,  1.8570193 ,  3.5165267 ,  1.0719959 ,  0.7293594 ,
         2.3112679 ,  0.84064364,  2.1612198 ,  3.0060468 ,  1.9224309 ,
         1.1812272 ,  1.891209  ,  2.472405  ,  1.2888657 ,  1.6927787 ,
         2.1506999 ,  1.3459386 ,  2.0038981 ,  1.5

### 1.2 Guardar Embedding para uso futuro (Varisas imágenes)

In [4]:
#Aplicación del modelo a más de una imagen

#Rutas finales de las imágenes a procesar

imagen2 = ruta_imagenes_train + '998892.jpeg'
imagen3 = ruta_imagenes_train + '994535.jpeg'
#Lista para guardar todas las imágenes
imagen_array = [imagen2, imagen3]
#Método para guardar los embeddings de cada imagen en la misma carpeta de donde vienen las imágenes
openl3.process_image_file(imagen_array, batch_size = 32)

openl3: Processing /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/998892.jpeg (1/2)
openl3: /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/998892.npz exists, skipping.
openl3: Processing /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/994535.jpeg (2/2)
openl3: /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/994535.npz exists, skipping.


In [5]:
#En este método se leerán los embeddings de las imágenes generadas en la sección anterior

#Cargamos la data (embedding) de la imagen especificada
data = np.load(ruta_imagenes_train + '998892.npz')
#Obtenemos solo el embedding
emb = data['embedding']

emb[0]

array([-0.17171964,  2.7841794 ,  0.58611256, ...,  0.55718577,
        1.9009621 ,  1.2161709 ], dtype=float32)

In [6]:
#En esta sección se aplica el embedding a 10 imágenes distintas dentro de la carpeta de train images de manera directa

#Obtenemos la lista con los nombres (id's) de las imágenes de entrenamiento
train_file_names = os.listdir(ruta_imagenes_train)
#Nos quedamos solo con las primeras 10 imágenes
train_file_names = train_file_names[:10]

#Concatenamos el resto de la ruta del archivo al nombre de cada imagen
train_complete_file_names = [ruta_imagenes_train + x for x in train_file_names]

#Método para guardar los embeddings de cada imagen en la misma carpeta de donde vienen las imágenes
openl3.process_image_file(train_complete_file_names, batch_size = 32)

#Empezamos con el proceso de cargar los embeddings de cada imagen

#Lista en donde se guardará cada embedding
embs_files = list()
#Loop para ir guardando cada embedding
for imagen in train_file_names:
    #Cargamos la data (embedding) de la imagen especificada
    data = np.load(ruta_imagenes_train + imagen[:imagen.find('.')] + '.npz')
    #Obtenemos solo el embedding
    embs_files.append(data['embedding'][0])

embs_files[0]

openl3: Processing /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/174383279.jpeg (1/10)
openl3: /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/174383279.npz exists, skipping.
openl3: Processing /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/194747578.jpeg (2/10)
openl3: /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/194747578.npz exists, skipping.
openl3: Processing /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/196588153.jpeg (3/10)
openl3: /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/196588153.npz exists, skipping.
openl3: Processing /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/8324721.jpeg (4/10)
openl3: /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/8324721.npz exists, skipping.
openl3: Processing /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/179983287.jpeg (5/10)
openl3: /Users/pedrovela/Docs/Datasets - ML/planttrait

array([-0.47553322,  2.9182515 , -0.14319089, ...,  0.74044186,
        2.1243396 , -0.49356037], dtype=float32)

### 1.3 Uso del modelo pre cargado

In [7]:
#Método en el cual generamos el embedding de la imagen ya con el modelo pre cargado

#Obtenemos el embedding final
emb = openl3.get_image_embedding(imagen1, model=modelo)
emb

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step


array([[-0.20485121,  2.7495792 ,  1.1525512 ,  1.6354275 ,  1.5630054 ,
         3.0043402 ,  1.1680847 ,  0.20437273,  0.47765186,  1.6069542 ,
         0.99042135,  2.744746  ,  3.0113118 ,  0.697831  ,  2.5043488 ,
         2.8977034 ,  2.1103582 ,  1.0919865 ,  2.5187943 ,  2.6752896 ,
         2.5137546 ,  0.80452126,  0.9297262 ,  2.2770233 ,  2.8381968 ,
         0.6253177 ,  0.7834051 ,  1.436738  ,  1.1495699 ,  1.3403784 ,
         1.7564527 ,  1.0250019 ,  2.2259736 ,  0.5685906 ,  2.5769222 ,
         0.8931727 ,  2.3390589 ,  1.2697175 ,  1.2542069 ,  1.3876815 ,
         1.3700166 ,  1.7157243 ,  0.76253283,  1.8189112 ,  0.24554229,
         1.335274  ,  1.7735906 ,  1.3587192 ,  1.4913703 ,  1.041074  ,
         0.53341097,  0.9961289 ,  0.8008581 ,  1.6766714 ,  1.8453351 ,
         1.4003036 ,  1.7122384 ,  1.1727496 ,  1.851693  ,  2.0431597 ,
         2.2497199 ,  1.0162674 ,  2.1898563 ,  1.1334101 ,  0.64272827,
         1.9252778 ,  2.6810825 ,  1.1118598 ,  2.4

## Modelo 2: ViT (Google)

[GitHub](https://github.com/lukemelas/PyTorch-Pretrained-ViT?tab=readme-ov-file) , [Colab](https://colab.research.google.com/drive/1muZ4QFgVfwALgqmrfOkp7trAvqDemckO?usp=sharing)

Papers de referencia:

- [AN IMAGE IS WORTH 16X16 WORDS: TRANSFORMERS FOR IMAGE RECOGNITION AT SCALE](https://openreview.net/pdf?id=YicbFdNTTy)

- [MLP-Mixer: An all-MLP Architecture for Vision](https://arxiv.org/pdf/2105.01601)
 
- [How to train your ViT? Data, Augmentation, and Regularization in Vision Transformers](https://arxiv.org/pdf/2106.10270)
   
- [Attention Is All You Need](https://arxiv.org/pdf/1706.03762v7)

Proceso general del modelo:

1. Considera una imagen como una secuencia mono-dimensional (1 dimensión) de parches (token embeddings)
  
2. Antepone un token de clasificación a la secuencia generada en el punto 1

3. Pone estos parches a través de un codificador transformador (como BERT)

4. Pasa el primer token del output generado por el transformador a través de un pequeño MLP (Multilayer Perceptron) para obtener los logits de clasificación. Deja una capa escondida para pre-entrenamiento y una sola capa lineal para el fine-tunning

Puntos sobre la lectura:

- Se divide una imagen en parches y provee la secuencia de embeddings lineales de esos parches como entrada a un Transformador

- Los parches son tratados como Tokens (palabras) en una aplicación NLP

- El encoder Transformer consiste en alternar capas de multiheaded self-attention (MSA) y bloques MLP

- En este modelo existen dos tipos de capas: las capas MLP y MSA

- La longitud de la secuencia del Transformador es inversamente proporcional al cuadrado del tamaño del parche, por lo que modelos con tamaños pequeños de parches son computacionalmente más caros

- Aquí también se utiliza el optmizador Adam (la otra alternativa es SGD)

- **INVESTIGAR EPOCHS, LEARNING RATE WARMUP, SGD, FLOPs**

- fine-tunning con SGD con momentum de 0.9, el fine-tunning general del modelo se hace con una resolución de 384 (a menos que se estipule otra cosa). EL Vision Transformer se beneficia con mejores resoluciones.

Rápida descripción de la inspección interna de ViT:

1. La primera capa de ViT proyecta linealmente los parches aplanados a un espacio con dimensión menor

2. Se agrega una posición de embedding a cada parche. El modelo aprende a calcular la distancia (similitud) entre las imagenes con respecto a su posición de embedding; parches cercanos tienden a tener posiciones de embeddings más similares. Esta distancia es de coseno entre la posición del embedding del parche de un determinado renglón y columna y la posición del embedding de los demás parches. Parches que se encuentran en la misma columna o renglón tienen embeddings similares

3. Una estructura sinusoidal (de tipo seno) se utiliza típicamente para mallas grandes

4. Self-attention permite a ViT integrar información a través de toda la imagen, incluso en las capas más bajas. Es una distancia que se mide entre el pixel de consulta (query) y los demás pixeles, todas con un peso de atención (weight). Mientras la capa sea más baja (profunda), la atención que se le da a la imagen es mayor.

5. El modelo atiende a regiones de la imagen que son semánticamente relevantes para la clasificación


**TRATAR DE REPLICAR ESTOS EJEMPLOS:** [Kaggle Example](https://www.kaggle.com/code/raufmomin/vision-transformer-vit-from-scratch) [Medium Example](https://medium.com/@diego.machado/fine-tuning-vit-for-image-classification-with-hugging-face-48c4be31e367) [Busqueda de Google (referencias)](https://www.google.com/search?q=using+vit+model+with+csv+file&client=safari&sca_esv=41a53a54b5bc3fc7&sca_upv=1&sxsrf=ADLYWIL7bNtclYzSlPXpYVpatUTD_zwpYw%3A1723677174417&source=hp&ei=9jm9Zry5F-7HkPIPiLy8iAo&iflsig=AL9hbdgAAAAAZr1IBha6iJ9JBpkWgf9h1nv91qK1KZIl&ved=0ahUKEwi8jL_tzfWHAxXuI0QIHQgeD6EQ4dUDCBY&uact=5&oq=using+vit+model+with+csv+file&gs_lp=Egdnd3Mtd2l6Ih11c2luZyB2aXQgbW9kZWwgd2l0aCBjc3YgZmlsZTIFECEYoAFIsj9QAFjZPnAAeACQAQCYAZ0BoAG_GKoBBDYuMjK4AQPIAQD4AQGYAhugArIYwgIKECMYgAQYJxiKBcICBBAjGCfCAg4QABiABBixAxiDARiKBcICERAuGIAEGLEDGNEDGIMBGMcBwgILEC4YgAQYsQMYgwHCAgsQABiABBixAxiDAcICBRAAGIAEwgIREC4YgAQYsQMYxwEYjgUYrwHCAggQLhiABBixA8ICCBAAGIAEGLEDwgIFEC4YgATCAg4QLhiABBjHARiOBRivAcICChAAGIAEGLEDGArCAgsQLhiABBjHARivAcICDRAAGIAEGLEDGEYY_wHCAgYQABgWGB7CAgcQABiABBgTwgIJEAAYgAQYExgKwgIIEAAYExgWGB7CAgQQIRgVmAMAkgcEMy4yNKAH734&sclient=gws-wiz)

In [2]:
#Librerias a importar

#!pip3 install pytorch_pretrained_vit

from pytorch_pretrained_vit import ViT
model = ViT('B_16_imagenet1k', pretrained=True)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 29.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 20.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 4.5 MB/s eta 0:00:00
  Created wheel for pytorch_pretrained_vit: filename=pytorch_pretrained_vit-0.0.7-py3-none-any.whl size=11114 sha256=9f5a18f9e5c23c5849b1b1a6b48aaaf0e5b85aafac1e211526c97205ad4ea5e6
  Stored in directory: /Users/pedrovela/Library/Caches/pip/wheels/79/c8/4f/9ad72c6f247a7e6888cc7767db9632675cf82656fffec85518
Successfully built pytorch_pretrained_vit


Downloading: "https://github.com/lukemelas/PyTorch-Pretrained-ViT/releases/download/0.0.2/B_16_imagenet1k.pth" to /Users/pedrovela/.cache/torch/hub/checkpoints/B_16_imagenet1k.pth
100%|████████████████████████████████████████| 331M/331M [00:08<00:00, 38.7MB/s]


Loaded pretrained weights.


In [7]:
#!pip3 install torchvision

import json
from PIL import Image
import torch
from torchvision import transforms

from functions import general_functions as gf

ruta_imagenes_train = gf.get_data_path('train_images')
img_name = ruta_imagenes_train + '993123.jpeg'

# Load ViT
from pytorch_pretrained_vit import ViT
model = ViT('B_16_imagenet1k', pretrained=True)
model.eval()

# Load image
# NOTE: Assumes an image `img.jpg` exists in the current directory
img = transforms.Compose([
    transforms.Resize((384, 384)), 
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
])(Image.open(img_name)).unsqueeze(0)
print(img.shape) # torch.Size([1, 3, 384, 384])

# Classify
with torch.no_grad():
    outputs = model(img).squeeze(0)
print(outputs.shape)  # (1, 1000)

Loaded pretrained weights.
torch.Size([1, 3, 384, 384])
torch.Size([1000])


In [8]:
outputs

tensor([ 4.1168e-02, -3.6647e-01, -8.2826e-02, -5.2003e-03, -6.9432e-01,
         6.3112e-01,  1.1538e-01, -9.5176e-01, -2.8889e-01,  1.1204e-01,
         9.6513e-01, -3.8003e-01, -2.1552e-01, -6.1709e-01,  8.0917e-01,
        -2.5889e-01,  5.4547e-02, -1.1185e+00, -5.8974e-01, -3.2512e-01,
        -2.5198e-01,  5.5554e-01, -2.1421e-02,  3.8290e-01, -2.5087e-02,
         1.9608e+00,  3.3388e+00,  2.1743e+00,  1.2444e+00,  8.5103e-01,
         9.4214e-01,  1.1039e+00,  2.9827e+00, -1.2655e+00,  3.1554e-01,
         9.3914e-01,  8.1178e-01,  4.6449e-01,  2.8589e+00,  3.0353e-01,
         1.3648e+00,  3.1621e+00,  7.6384e-01,  1.6017e+00,  2.9447e+00,
         1.9694e+00,  1.7545e+00,  2.4520e+00, -8.3982e-03,  6.9470e-02,
        -5.9563e-01, -4.6310e-01,  3.2046e+00,  2.7494e+00,  2.3458e+00,
         1.7325e+00,  2.8559e+00,  1.9144e+00,  9.2152e-01,  2.1196e+00,
         1.6230e+00,  9.6469e-01,  1.4489e+00,  1.3848e+00,  1.8677e+00,
         6.0899e-01,  1.9174e+00,  2.1616e+00,  2.7

In [11]:
torch.topk(outputs, k=4).indices.tolist()

[322, 311, 70, 320]

In [12]:
torch.softmax(outputs, -1)[322].item()

0.04724839702248573